In [1]:
import os

import mala
from mala import printout
from mala.network import TesterGraph

In [2]:
parameters = mala.Parameters()

# Currently, the splitting in training, validation and test set are
# done on a "by snapshot" basis.
parameters.data.data_splitting_type = "by_snapshot"
parameters.data.use_graph_data_set = True

# Specify the training parameters.
parameters.running.max_number_epochs = 10
parameters.running.ldos_grid_batch_size = 200
parameters.running.learning_rate = 0.00001
parameters.running.trainingtype = "Adam"
parameters.running.visualisation = 1

parameters.targets.ldos_gridsize = 201
parameters.verbosity = 1

parameters.use_gpu = True

In [3]:
data_handler = mala.DataHandlerGraph(parameters)

# for i in range(16):
for i in range(1):
    data_handler.add_raw_snapshot(
        f'/bigdata/casus/wdm/Bartek_H2/H128/snapshot{i}/H_snapshot{i}.pw.scf.in',
        f'/bigdata/casus/wdm/Bartek_H2/H128/ldos/H_snapshot{i}.out.npy',
        (90, 90, 60, 201), 'tr'
    )
# for i in range(16, 18):
for i in range(1, 2):
    data_handler.add_raw_snapshot(
        f'/bigdata/casus/wdm/Bartek_H2/H128/snapshot{i}/H_snapshot{i}.pw.scf.in',
        f'/bigdata/casus/wdm/Bartek_H2/H128/ldos/H_snapshot{i}.out.npy',
        (90, 90, 60, 201), 'va'
    )

data_handler.prepare_data(reparametrize_scaler=False)

No data rescaling will be performed.
No data rescaling will be performed.


AttributeError: 'DataHandlerGraph' object has no attribute 'add_raw_snapshot'

In [ ]:
parameters.network.nn_type = "se3_transformer"
parameters.network.layer_sizes = [
    data_handler.input_dimension,
    16,
    data_handler.output_dimension
]
# Setup network and trainer.
network = mala.Network(parameters)
trainer = mala.TrainerGraph(parameters, network, data_handler)
trainer.train_network()

Writing visualization output to ./mala_logging/2023-05-30-19-01-11
Validating ldos on validation data set.


  0%|          | 0/2430 [00:00<?, ?it/s]/home/brzoza73/casus/mala-venv/lib/python3.8/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/home/brzoza73/casus/mala-venv/lib/python3.8/site-packages/dgl/backend/pytorch/tensor.py:445: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (
100%|██████████| 2430/2430 [00:31<00:00, 76.65it/s]


Initial Guess - validation data loss:  0.009408645472899387
Validating ldos on validation data set.


100%|██████████| 2430/2430 [00:30<00:00, 78.70it/s]


Epoch 0: validation data loss: 0.002713410452069569
Validating ldos on validation data set.


...

Epoch 7: validation data loss: 0.0006849766758734307
Validating ldos on validation data set.


100%|██████████| 2430/2430 [00:30<00:00, 78.69it/s]


Epoch 8: validation data loss: 0.0006834000716974706
Validating ldos on validation data set.


100%|██████████| 2430/2430 [00:30<00:00, 78.83it/s]

Epoch 9: validation data loss: 0.000685942025832188
Final validation data loss:  0.000685942025832188


In [7]:
data_handler_test = mala.DataHandlerGraph(parameters)

# for i in range(16):
for i in range(2, 3):
    data_handler_test.add_raw_snapshot(
        f'/bigdata/casus/wdm/Bartek_H2/H128/snapshot{i}/H_snapshot{i}.pw.scf.in',
        f'/bigdata/casus/wdm/Bartek_H2/H128/ldos/H_snapshot{i}.out.npy',
        (90, 90, 60, 201), 'te'
    )

data_handler_test.prepare_data(reparametrize_scaler=False)


No data rescaling will be performed.
No data rescaling will be performed.
Checking the snapshots and your inputs for consistency.
DataHandler prepared for inference. No training possible with this setup. If this is not what you wanted, please revise the input script. Validation snapshots you may have entered willbe ignored.
Consistency check successful.
Build datasets.
Build dataset: Done.


In [9]:
network = network.to('cuda')
observables_to_test = [
  "ldos",
  "band_energy",
  "band_energy_full",
  "number_of_electrons",
  "total_energy",
  "total_energy_full",
  "density",
  "dos"
]

tester = TesterGraph(parameters, network, data_handler_test, observables_to_test=observables_to_test)
tester.test_all_snapshots()

# have to input somethin into
# tester.parameters.snapshot_directories_list

100%|██████████| 2430/2430 [00:51<00:00, 47.63it/s]


IndexError: list index out of range

In [11]:
trainer.save_run("H2_model")

In [2]:
import torch
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |